<a href="https://colab.research.google.com/github/fritzmartin003/RAG-System-Projekt/blob/main/HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf openai numpy scipy


In [ ]:
#mit Secret Manager in Google Colab
from google.colab import userdata
userdata.get('HF')

In [17]:
pdf_path = "SakowskiBuch.pdf"

In [18]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

# Text aus PDF extrahieren
pdf_text = extract_text_from_pdf(pdf_path)
print(" Text aus PDF extrahiert!")


 Text aus PDF extrahiert!


In [19]:
def split_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap  # Überschneidung
    return chunks

chunks = split_text(pdf_text)
print(f" PDF in {len(chunks)} Chunks unterteilt!")


 PDF in 1559 Chunks unterteilt!


In [20]:
import numpy as np
import openai
import os

In [ ]:
!pip install sentence-transformers

In [ ]:

from sentence_transformers import SentenceTransformer

# Hugging Face Embedding-Modell laden
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Funktion zum Erstellen von Embeddings
def get_embedding(text):
    return embedding_model.encode(text, convert_to_numpy=True)

# Alle Chunks in Embeddings umwandeln
chunk_embeddings = np.array([get_embedding(chunk) for chunk in chunks])

print("✅ Chunks als Vektoren gespeichert!")


In [22]:
from scipy.spatial.distance import cosine

def get_relevant_chunks(query, top_k=3):
    query_embedding = get_embedding(query)

    # Cosine Similarity berechnen
    similarities = [1 - cosine(query_embedding, chunk_emb) for chunk_emb in chunk_embeddings]

    # Top-k Chunks auswählen (die mit der höchsten Ähnlichkeit)
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [chunks[i] for i in top_indices]

print(" Ähnlichkeitssuche bereit!")


 Ähnlichkeitssuche bereit!
